In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset, Dataset
import yfinance as yf
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

warnings.filterwarnings('ignore')

In [20]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [21]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=2, output_size=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers,
            batch_first=True, dropout=0.2
        )
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        device = x.device

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out


In [23]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim, 1)

    def forward(self, lstm_out):
        weights = torch.softmax(self.attn(lstm_out), dim=1)
        context = torch.sum(weights * lstm_out, dim=1)
        return context


class CNN_BiLSTM_Attention(nn.Module):
    def __init__(self, input_size, hidden_size, cnn_out=32, kernel=3):
        super().__init__()

        self.conv = nn.Conv1d(input_size, cnn_out, kernel, padding=kernel // 2)
        self.bn = nn.BatchNorm1d(cnn_out)
        self.relu = nn.ReLU()

        self.lstm = nn.LSTM(
            input_size=cnn_out,
            hidden_size=hidden_size,
            batch_first=True,
            bidirectional=True
        )

        self.attention = Attention(hidden_dim=hidden_size * 2)
        self.fc = nn.Linear(hidden_size * 2, 1)

    def forward(self, x):
        x = x.permute(0, 2, 1)           # (batch, features, seq)
        x = self.relu(self.bn(self.conv(x)))
        x = x.permute(0, 2, 1)

        lstm_out, _ = self.lstm(x)
        context = self.attention(lstm_out)
        output = self.fc(context)
        return output


In [24]:
def normalize_yf_dataframe(df):
    df = df.copy()
    
    if isinstance(df.columns, pd.MultiIndex):
        df.columns.names = ['DataType', 'Ticker']
        df = df.stack(level=1).reset_index()
        df = df.rename(columns={'level_1': 'Ticker'})
    else:
        df = df.reset_index()
        df['Ticker'] = df.columns[1].split()[0] if ' ' in df.columns[1] else 'UNKNOWN'

    df = df.rename(columns=str.capitalize)
    cols = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']
    df = df[[c for c in cols if c in df.columns]]

    return df


In [25]:
def make_sliding_window(data, feature_cols, target_col, window_size=60):
    X, y = [], []
    values = data[feature_cols].values
    targets = data[target_col].values

    if len(data) <= window_size:
        raise ValueError(f"Недостаточно данных для построения sliding window: "
                         f"len(data)={len(data)}, window_size={window_size}")

    for i in range(window_size, len(data)):
        X.append(values[i - window_size:i])
        y.append(targets[i])
    
    return np.array(X), np.array(y)


In [26]:
def scale(data, scaler):
    csale_d = data.reshape(-1, data.shape[2])
    csale_d = scaler.transform(csale_d)
    return csale_d.reshape(data.shape)

In [27]:
def prepare_data(df, seq_len, shift):
    df = normalize_yf_dataframe(df)

    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])

    target_col = f"Next_Close_t{shift}"
    df[target_col] = df["Close"].shift(-shift)
    df = df.dropna().reset_index(drop=True)

    features = ["Open", "High", "Low", "Close", "Volume"]
    features = [ "Close"]

    # ==== масштабування до sliding window ====
    X_raw = df[features].values           # (N, 5)
    y_raw = df[[target_col]].values       # (N, 1)

    scaler_x = StandardScaler().fit(X_raw)
    scaler_y = StandardScaler().fit(y_raw)

    df_scaled = df.copy()
    df_scaled[features] = scaler_x.transform(X_raw)
    df_scaled[target_col] = scaler_y.transform(y_raw)

    # ==== sliding window ====
    X, y = make_sliding_window(df_scaled, features, target_col, window_size=seq_len)

    # ==== split ====
    train_size = int(len(X) * 0.7)
    val_size = int(len(X) * 0.15)

    X_train = X[:train_size]
    y_train = y[:train_size]

    X_val = X[train_size:train_size + val_size]
    y_val = y[train_size:train_size + val_size]

    X_test = X[train_size + val_size:]
    y_test = y[train_size + val_size:]

    # ==== torch loaders ====
    train_loader = DataLoader(
        TimeSeriesDataset(X_train, y_train),
        batch_size=32, shuffle=True
    )

    val_loader = DataLoader(
        TimeSeriesDataset(X_val, y_val),
        batch_size=32, shuffle=False
    )

    return (
        train_loader,
        val_loader,
        X_test,
        y_test,
        scaler_x,
        scaler_y,
        df_scaled
    )


In [28]:
def train(model, loader, val_loader, optimizer, loss_fn, device, epochs=30):
    # scheduler снижает LR, если val_loss перестал улучшаться
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=3
    )

    for epoch in range(epochs):
        model.train()
        losses = []

        for Xb, yb in loader:
            Xb, yb = Xb.to(device), yb.to(device)

            optimizer.zero_grad()
            pred = model(Xb).flatten()

            loss = loss_fn(pred, yb)
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

        # validation
        model.eval()
        val_losses = []
        with torch.no_grad():
            for Xb, yb in val_loader:
                Xb, yb = Xb.to(device), yb.to(device)
                pred = model(Xb).flatten()
                val_losses.append(loss_fn(pred, yb).item())

        train_loss = np.mean(losses)
        val_loss = np.mean(val_losses)

        # шаг шедулера
        scheduler.step(val_loss)

        print(
            f"Epoch {epoch+1}/{epochs} | "
            f"Train Loss={train_loss:.4f} | "
            f"Val Loss={val_loss:.4f} | "
            f"LR={optimizer.param_groups[0]['lr']:.6f}"
        )


In [29]:
def evaluate(model, X, y_true_scaled, scaler_y, device):
    model.eval()
    X = torch.tensor(X, dtype=torch.float32).to(device)

    with torch.no_grad():
        preds = model(X).cpu().numpy().flatten()

    # вернуть масштаб
    preds_real = scaler_y.inverse_transform(preds.reshape(-1, 1)).flatten()
    y_real = scaler_y.inverse_transform(y_true_scaled.reshape(-1, 1)).flatten()

    mae = mean_absolute_error(y_real, preds_real)
    rmse = mean_squared_error(y_real, preds_real) ** 0.5
    mape = np.mean(np.abs((y_real - preds_real) / y_real)) * 100
    r2 = r2_score(y_real, preds_real)

    print("\n=== METRICS ===")
    print(f"MAE:  {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"R²:   {r2:.4f}")

    return preds_real, y_real

In [30]:
def pipeline(df, seq_len, shift, epochs):
    train_loader, val_loader, X_test, y_test_sc, scaler_x, scaler_y,df = prepare_data(df, seq_len, shift)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    # device ="cpu"
    print("Using device:", device)

    model = CNN_BiLSTM_Attention(input_size=X_test.shape[2], hidden_size=64).to(device)
    model = LSTMModel(input_size=X_test.shape[2], hidden_size=64)

    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    model = model.to(device)

    train(model, train_loader, val_loader, optimizer, loss_fn, device, epochs)
    preds, y_real = evaluate(model, X_test, y_test_sc, scaler_y, device)

    return model, scaler_x, scaler_y, optimizer




In [15]:
ticker = "MSFT"

df = yf.download(ticker, period="2y", interval="1h")

[*********************100%***********************]  1 of 1 completed


In [17]:

df = yf.download( "BTC-USD", period="2y", interval="1h")

[*********************100%***********************]  1 of 1 completed


In [18]:
df.index = df.index.tz_convert('Europe/Kiev')

df

Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Datetime,,,,,
2023-11-21 19:00:00+02:00,36947.367188,37504.339844,36761.171875,37189.367188,0
2023-11-21 20:00:00+02:00,36906.851562,36974.835938,36747.679688,36917.785156,0
2023-11-21 21:00:00+02:00,36991.433594,37109.394531,36779.968750,36901.960938,61929472
2023-11-21 22:00:00+02:00,36963.566406,37167.101562,36866.132812,36983.945312,442103808
2023-11-21 23:00:00+02:00,36815.226562,37025.535156,36750.574219,36938.578125,82348032
...,...,...,...,...,...
2025-11-21 14:00:00+02:00,83142.976562,83471.179688,80756.515625,82217.890625,8844345344
2025-11-21 15:00:00+02:00,84121.773438,84270.007812,83142.187500,83152.867188,2319384576


In [19]:
res = pipeline(df, 60, 8, 60)

NameError: name 'pipeline' is not defined

In [31]:
def create_sequences(X, window):
    X_seq = []
    for i in range(len(X) - window):
        X_seq.append(X[i:i+window])
    return np.array(X_seq)

In [33]:
def make_predictions(df, model, scaler_x, scaler_y, window, col_name, device):
    model.eval()

    features = ["Close"]
    X_raw = df[features].values

    X_all_scaled = scaler_x.transform(X_raw)

    X_seq = []
    for i in range(len(X_all_scaled) - window):
        X_seq.append(X_all_scaled[i:i+window])
    X_seq = np.array(X_seq)

    X_tensor = torch.tensor(X_seq, dtype=torch.float32).to(device)

    with torch.no_grad():
        y_pred_scaled = model(X_tensor).cpu().numpy().flatten()

    y_pred_real = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

    pred_column = np.full(len(df), np.nan)
    start_index = window
    pred_column[start_index:start_index + len(y_pred_real)] = y_pred_real

    df[col_name] = pred_column
    return df


In [34]:
import joblib 
import os


def create_models(df: pd.DataFrame, shifts=(1, 3, 8), path_dir=None):
    save_dir = 'models'
    df_new = df.copy()
    os.makedirs(save_dir, exist_ok=True)
    if path_dir:
        save_dir = f'models/{save_dir}'
        os.makedirs(save_dir, exist_ok=True)


    for shift in shifts:
        path = f'predict_t{shift}'
        os.makedirs(os.path.join(save_dir, path), exist_ok=True)

        model, scaler_x, scaler_y, optimizer = pipeline(df, 60, shift, 60)

        model_path = os.path.join(save_dir, path, "model.pth")
        scaler_x_path = os.path.join(save_dir, path, "scaler_x.pkl")
        scaler_y_path = os.path.join(save_dir, path, "scaler_y.pkl")

        torch.save({
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict()
        }, model_path)

        joblib.dump(scaler_x, scaler_x_path)
        joblib.dump(scaler_y, scaler_y_path)

        df_new = make_predictions(df_new, model, scaler_x, scaler_y, 60, path, 'cuda')

        print(f"✅ Модель t={shift} сохранена и предсказания добавлены")

    df_new.to_csv(os.path.join(save_dir, 'new_data.csv'), index=False)
    print("✅ Все модели обучены и результаты сохранены в new_data.csv")


In [35]:
create_models(df, path_dir= "BTC-USD")

Using device: cuda
Epoch 1/60 | Train Loss=0.0389 | Val Loss=0.0186 | LR=0.001000
Epoch 2/60 | Train Loss=0.0063 | Val Loss=0.0259 | LR=0.001000
Epoch 3/60 | Train Loss=0.0056 | Val Loss=0.0057 | LR=0.001000
Epoch 4/60 | Train Loss=0.0055 | Val Loss=0.0042 | LR=0.001000
Epoch 5/60 | Train Loss=0.0056 | Val Loss=0.0069 | LR=0.001000
Epoch 6/60 | Train Loss=0.0052 | Val Loss=0.0040 | LR=0.001000
Epoch 7/60 | Train Loss=0.0049 | Val Loss=0.0100 | LR=0.001000
Epoch 8/60 | Train Loss=0.0048 | Val Loss=0.0127 | LR=0.001000
Epoch 9/60 | Train Loss=0.0044 | Val Loss=0.0038 | LR=0.001000
Epoch 10/60 | Train Loss=0.0048 | Val Loss=0.0054 | LR=0.001000
Epoch 11/60 | Train Loss=0.0044 | Val Loss=0.0036 | LR=0.001000
Epoch 12/60 | Train Loss=0.0044 | Val Loss=0.0094 | LR=0.001000
Epoch 13/60 | Train Loss=0.0042 | Val Loss=0.0063 | LR=0.001000
Epoch 14/60 | Train Loss=0.0042 | Val Loss=0.0126 | LR=0.001000
Epoch 15/60 | Train Loss=0.0042 | Val Loss=0.0066 | LR=0.000500
Epoch 16/60 | Train Loss=0.004

In [136]:
df

Price,Close,High,Low,Open,Volume
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA
Datetime,,,,,
2023-11-13 14:30:00+00:00,219.985001,220.419998,211.610107,215.600006,38837399
2023-11-13 15:30:00+00:00,223.529999,223.919998,219.369995,219.970001,26384434
2023-11-13 16:30:00+00:00,224.130005,225.399994,222.710007,223.550003,20160326
2023-11-13 17:30:00+00:00,222.121307,224.330002,221.570007,224.101196,14999487
2023-11-13 18:30:00+00:00,224.860001,224.889999,221.770004,222.119995,13243605
...,...,...,...,...,...
2025-11-11 14:30:00+00:00,435.266510,442.489990,434.820007,439.399994,13387275
2025-11-11 15:30:00+00:00,433.369995,436.399994,432.739990,435.320007,8678369


Начало обучения моделей для разных горизонтов предсказания...

=== Обучение модели для предсказания на 1 час(ов) ===
Ошибка при обучении модели для 1 часа(ов): Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by StandardScaler.

=== Обучение модели для предсказания на 3 час(ов) ===
Ошибка при обучении модели для 3 часа(ов): Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by StandardScaler.

=== Обучение модели для предсказания на 9 час(ов) ===
Ошибка при обучении модели для 9 часа(ов): Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by StandardScaler.

DataFrame с предсказаниями сохранен: saved_models\predictions_results_20251111_2105.csv
